In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd


path = "/content/drive/My Drive/fire_predict/base_data/"
df_train = pd.read_csv(path + "PJT002_train.csv")
df_val = pd.read_csv(path + "PJT002_validation.csv")
df_test = pd.read_csv(path + "PJT002_test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


def f_data(df_train, df_val, df_test):

    X_train = df_train.drop(['fr_yn'], 1)
    y_train = df_train['fr_yn']
    X_val = df_val.drop(['fr_yn'], 1)
    y_val = df_val['fr_yn']
    X_test = df_test

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(df_train)]
    X_val = df_all[len(df_train):-len(df_test)]
    X_test = df_all[-len(df_test):]

    X_train = X_train.fillna(-1)
    X_val = X_val.fillna(-1)
    X_test = X_test.fillna(-1)

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(bootstrap=True, max_depth=80,
                                       max_features="auto", min_samples_leaf=4,
                                       min_samples_split=5, n_estimators=200,
                                       random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

def f1_rfc_2(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(random_state=42, n_estimators=1000)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

# lable -> binary
binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

#df ->df_2
df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

#drop id
df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

#drop dt_of_fr
df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

#dt_of_athrztn -> year 
def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])

df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

# hm_cnt -> categorical
def population(x):
    if x < 1000:
        return "1k"
    elif x < 10000:
        return "10k"
    elif x < 100000:
        return "100k"
    elif x < 1000000:
        return "1000k"

df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

#tmprtr c -> k
df_train_2.tmprtr = df_train_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

#gas_engry_us_201507 x < 0 -> remove
df_train_2.gas_engry_us_201507 = df_train_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

# drop gas, elec
df_train_2 = df_train_2.drop(df_train_2.iloc[:,16:136].columns, axis=1)


X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
print(f1_rfc_2(X_train, y_train, X_val, y_val))
print(f1_rfc(X_train, y_train, X_val, y_val))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  0%|          | 0/10 [00:00<?, ?it/s]

0.4879288437102921


100%|██████████| 10/10 [03:24<00:00, 20.36s/it]

0.481675392670157


In [0]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


def f_data(train, val, test):

    X_train = train.drop(['fr_yn'], 1)
    y_train = train['fr_yn']
    X_val = val.drop(['fr_yn'], 1)
    y_val = val['fr_yn']
    X_test = test.drop(['fr_yn'], 1)

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(train)]
    X_val = df_all[len(train):-len(test)]
    X_test = df_all[-len(test):]

    # X_train = X_train.fillna(-1)
    # X_val = X_val.fillna(-1)
    # X_test = X_test.fillna(-1)

    X_train = X_train.replace({-1:np.NaN})
    X_val = X_val.replace({-1:np.NaN})
    X_test = X_test.replace({-1:np.NaN})

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(bootstrap=True, max_depth=80,
                                       max_features="auto", min_samples_leaf=4,
                                       min_samples_split=5, n_estimators=200,
                                       random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

def f1_rfc_2(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(random_state=42, n_estimators=1000)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

# lable -> binary
binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

#df ->df_2
df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

#drop id
df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

#drop dt_of_fr
df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

#dt_of_athrztn -> year 
def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])

df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

# hm_cnt -> categorical
def population(x):
    if x < 1000:
        return "1k"
    elif x < 10000:
        return "10k"
    elif x < 100000:
        return "100k"
    elif x < 1000000:
        return "1000k"

df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

#tmprtr c -> k
df_train_2.tmprtr = df_train_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_val_2.tmprtr = df_val_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_test_2.tmprtr = df_test_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

#gas_engry_us_201507 x < 0 -> remove
df_train_2.gas_engry_us_201507 = df_train_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

# drop gas, elec
gas_elec_columns = list(df_train_2.iloc[:,16:136].columns)
df_train_2 = df_train_2.drop(gas_elec_columns, axis=1)
df_val_2 = df_val_2.drop(gas_elec_columns, axis=1)
df_test_2 = df_test_2.drop(gas_elec_columns, axis=1)

#for imputation
s = df_train_2.isnull().sum()
c_list = np.where(s >= max(s)*0.5)
df_train_2 = df_train_2.drop(list(s.index[c_list[0]]), axis=1)

df_val_2 = df_val_2.drop(list(s.index[c_list[0]]), axis=1)
df_test_2 = df_test_2.drop(list(s.index[c_list[0]]), axis=1)

X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)

In [0]:
!pip install impyute

In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS

# start the KNN training
X_train=fast_knn(X_train.values, k=3)

In [0]:
pd.DataFrame(X_train).to_csv(path + 'imputed_34f.csv',index=False, header=False)

In [0]:
X_train = pd.read_csv(path + "imputed_34f.csv", header=None)
X_val = X_val.fillna(-1)
X_test = X_test.fillna(-1)
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,9.000000,8.000000,3.0,69.420,69.4200,0.0,1977.000000,1.000000,0.000000,6.000000,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,12.649990,9.058762,3.0,46.290,46.2900,0.0,1984.654245,1.000000,0.000000,5.215133,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0
2,1.000000,15.000000,1.0,583.800,2516.7600,1446.0,2000.000000,5.000000,0.000000,6.000000,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0
3,9.000000,10.000000,2.0,48.920,48.9200,0.0,1936.000000,1.000000,0.000000,6.000000,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0
4,14.538810,6.531796,2.0,0.000,0.0000,0.0,1968.814380,1.000000,0.000000,5.692288,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.000000,1.000000,2.0,423.000,423.0000,3010.0,1996.000000,3.121446,0.083291,5.215133,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0
59195,12.649990,9.058762,35.0,5593.711,54835.0382,0.0,1984.654245,171.000000,0.000000,5.215133,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0
59196,5.407946,10.378813,2.0,2436.820,2468.8000,6512.0,2002.829298,4.000000,0.000000,2.348822,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,6.397596e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0
59197,27.000000,1.000000,2.0,283.200,207.7100,0.0,2005.000000,2.000000,0.000000,5.000000,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.397596e+05,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0


In [0]:
print(f1_rfc_2(X_train, y_train, X_val.values, y_val))
print(f1_rfc(X_train, y_train, X_val.values, y_val))



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [03:15<29:21, 195.77s/it]

 20%|██        | 2/10 [06:28<25:58, 194.78s/it]

 30%|███       | 3/10 [09:30<22:17, 191.05s/it]

 40%|████      | 4/10 [12:30<18:45, 187.58s/it]

 50%|█████     | 5/10 [15:30<15:26, 185.38s/it]

 60%|██████    | 6/10 [18:31<12:16, 184.11s/it]

 70%|███████   | 7/10 [21:31<09:08, 182.96s/it]

 80%|████████  | 8/10 [24:32<06:04, 182.16s/it]

 90%|█████████ | 9/10 [27:33<03:01, 181.93s/it]

100%|██████████| 10/10 [30:33<00:00, 181.44s/it]



  0%|          | 0/10 [00:00<?, ?it/s]

0.4673869547819128




 10%|█         | 1/10 [00:33<04:58, 33.21s/it]

 20%|██        | 2/10 [01:06<04:25, 33.18s/it]

 30%|███       | 3/10 [01:39<03:52, 33.17s/it]

 40%|████      | 4/10 [02:12<03:18, 33.13s/it]

 50%|█████     | 5/10 [02:45<02:45, 33.10s/it]

 60%|██████    | 6/10 [03:18<02:12, 33.11s/it]

 70%|███████   | 7/10 [03:51<01:39, 33.12s/it]

 80%|████████  | 8/10 [04:24<01:06, 33.12s/it]

 90%|█████████ | 9/10 [04:57<00:33, 33.09s/it]

100%|██████████| 10/10 [05:31<00:00, 33.12s/it]



0.4670807453416149


In [0]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
import numpy as np


# Number of trees in random forest
n_estimators = [200, 500, 1000]
# Number of features to consider at every split
max_features = ['auto', None]
# Maximum number of levels in tree
max_depth = [80]

# Minimum number of samples required to split a node
min_samples_split = [5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True],
 'max_depth': [80],
 'max_features': ['auto', None],
 'min_samples_leaf': [4],
 'min_samples_split': [5],
 'n_estimators': [200, 500, 1000]}


In [38]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, random_state=42)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 1000}